In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# read in files
data = pd.read_csv('data/data_result_feature_engineering.csv')
orders_new = pd.read_csv('data/orders_new.csv')
order_products_new = pd.read_csv('data/order_products_new.csv')

In [3]:
# merge data frames
orders_new_temp = orders_new.loc[:,['user_id','eval_set']]
orders_new_temp = orders_new_temp[orders_new_temp['eval_set'].isin(['train', 'test'])]
df = pd.merge(data, orders_new_temp, on = 'user_id', how = 'left')

In [4]:
df

,user_id,product_id,user_orders,user_period,user_mean_days_since_prior,user_total_products,user_distinct_products,user_reorder_ratio,up_orders,up_first_order,...,up_average_cart_position,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,eval_set
0,1,196,10,1094.0,20.259259,59,18,0.759259,10,1,...,1.400000,35791,0.582500,4.473875,0.776480,5.900000,1.000000,0,1.000000,train
1,1,10258,10,1094.0,20.259259,59,18,0.759259,9,2,...,3.333333,1946,0.552962,3.493716,0.713772,5.900000,0.900000,0,1.000000,train
2,1,10326,10,1094.0,20.259259,59,18,0.759259,1,5,...,5.000000,5526,0.521581,2.873635,0.652009,5.900000,0.100000,5,0.166667,train
3,1,12427,10,1094.0,20.259259,59,18,0.759259,10,1,...,3.300000,6476,0.529482,3.857058,0.740735,5.900000,1.000000,0,1.000000,train
4,1,13032,10,1094.0,20.259259,59,18,0.759259,3,2,...,6.333333,3751,0.479782,2.916796,0.657158,5.900000,0.300000,0,0.333333,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13307948,206209,43961,13,2352.0,20.275862,129,68,0.525862,3,4,...,8.000000,55371,0.514642,2.706967,0.630583,9.923077,0.230769,1,0.300000,train
13307949,206209,44325,13,2352.0,20.275862,129,68,0.525862,1,7,...,8.000000,3485,0.289890,1.669861,0.401148,9.923077,0.076923,6,0.142857,train
13307950,206209,48370,13,2352.0,20.275862,129,68,0.525862,1,11,...,8.000000,3934,0.592561,3.325444,0.699288,9.923077,0.076923,2,0.333333,train
13307951,206209,48697,13,2352.0,20.275862,129,68,0.525862,1,7,...,6.000000,9783,0.260662,1.556811,0.357661,9.923077,0.076923,6,0.142857,train


In [5]:
# train-test split
df_train = df[df['eval_set'] == 'train']
df_test = df[df['eval_set'] == 'Test']

# merge with order_products_new
orders_new_train = orders_new[orders_new['eval_set'] == 'train']
joint = pd.merge(orders_new_train, order_products_new, on = 'order_id', how = 'left')
joint = joint.loc[:,['user_id', 'product_id', 'reordered']]

# train set
train_set = pd.merge(df_train, joint, on = ['user_id', 'product_id'], how = 'left')
train_set.head()

,user_id,product_id,user_orders,user_period,user_mean_days_since_prior,user_total_products,user_distinct_products,user_reorder_ratio,up_orders,up_first_order,...,prod_orders,prod_reorder_probability,prod_reorder_times,prod_reorder_ratio,user_average_basket,up_order_rate,up_orders_since_last_order,up_order_rate_since_first_order,eval_set,reordered
0,1,196,10,1094.0,20.259259,59,18,0.759259,10,1,...,35791,0.582500,4.473875,0.776480,5.9,1.0,0,1.000000,train,1.0
1,1,10258,10,1094.0,20.259259,59,18,0.759259,9,2,...,1946,0.552962,3.493716,0.713772,5.9,0.9,0,1.000000,train,1.0
2,1,10326,10,1094.0,20.259259,59,18,0.759259,1,5,...,5526,0.521581,2.873635,0.652009,5.9,0.1,5,0.166667,train,NaN
3,1,12427,10,1094.0,20.259259,59,18,0.759259,10,1,...,6476,0.529482,3.857058,0.740735,5.9,1.0,0,1.000000,train,NaN
4,1,13032,10,1094.0,20.259259,59,18,0.759259,3,2,...,3751,0.479782,2.916796,0.657158,5.9,0.3,0,0.333333,train,1.0


In [6]:
train_set = train_set.drop(columns=['eval_set'])
test_set = df_test.drop(columns=['eval_set'])

In [9]:
# fill nan
train_set = train_set.fillna(0)
test_set = test_set.fillna(0)

train_features = train_set.iloc[:,:-1]
train_target = train_set.iloc[:,-1]

### XGBoost and K-fold cross validation

In [12]:
### k-fold cross validation functions

def train_validation_split(data, k):
    number_of_rows = data.shape[0]
    number_of_test = int(np.floor(number_of_rows / k))
    list_of_data_index = [i for i in range(number_of_rows)]
    
    result = pd.DataFrame(columns=['train_index','validation_index'], index=[i for i in range(k)])
    
    for i in range(k):
        total_index = np.array([i for i in range(number_of_rows)])
        test_index = np.random.choice(list_of_data_index, size=number_of_test, replace=False)
        train_index = np.delete(total_index, test_index)
        
        for index in test_index:
            list_of_data_index.remove(index)
        
        # print("{0}th iteration:\n".format({i}), "test index\n", test_index, "\ntrain index\n", train_index)
        
        result.iloc[i,0] = np.sort(train_index)
        result.iloc[i,1] = np.sort(test_index)

    return result



def combination_grid(parameters):
    import itertools
    '''
    parameters: a dictionary, e.g. parameters = {'max_depth':[3, 5, 7, 9], 'min_samples_leaf': [1, 2, 3, 4]}
    '''
    # https://www.codegrepper.com/code-examples/delphi/python+list+of+lists+all+combinations
    combinations = list(itertools.product(*list(parameters.values())))
    df = pd.DataFrame(combinations, columns = parameters.keys()) 
    
    return df

In [37]:
# spliting 10-fold train_validation

train_set_sample = train_set.sample(frac=0.005)

train_sample_features = train_set_sample.iloc[:,:-1]
train_sample_target = train_set_sample.iloc[:,-1]

split = train_validation_split(train_sample_features, 10)
split

,train_index,validation_index
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14,...","[12, 15, 32, 43, 67, 75, 88, 91, 120, 137, 148..."
1,"[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 15, 1...","[0, 8, 14, 37, 40, 58, 61, 69, 73, 81, 84, 94,..."
2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[24, 28, 31, 41, 44, 46, 55, 56, 71, 76, 82, 8..."
3,"[0, 1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15...","[3, 10, 16, 19, 23, 25, 33, 36, 47, 52, 54, 72..."
4,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 11, 12, 13, 14...","[9, 20, 29, 35, 39, 48, 53, 59, 60, 62, 66, 77..."
5,"[0, 1, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 1...","[2, 6, 17, 22, 34, 50, 83, 90, 98, 117, 118, 1..."
6,"[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15...","[1, 13, 42, 49, 51, 68, 79, 85, 93, 97, 103, 1..."
7,"[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15...","[5, 11, 18, 26, 30, 45, 64, 70, 80, 109, 113, ..."
8,"[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","[4, 38, 65, 87, 95, 111, 112, 114, 119, 126, 1..."
9,"[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14...","[7, 21, 27, 57, 63, 78, 99, 101, 107, 110, 121..."


In [38]:
# make grid of paramters

# booster [default=gbtree]
# slient [default=0]
# eta -- learning rate
eta = [0.01, 0.05, 0.15, 0.2]
# min_child_weight [default=1]
min_child_weight = [0.1,0.2,0.3, 0.4, 0.5, 1]
# max_depth
max_depth = [4, 6, 8]
# max_leaft_nodes
# gamma [default=0]
gamma = [0, 0.5]
# lambda [default=1]
# subsample [default=1]
subsample = [0.6, 0.75, 1]
# alpha [default=0]
# number_boost_round
number_boost_round = [500, 1000, 2000]
# early_stopping_rounds
early_stopping_rounds = [50]
# seed


# Create the random grid
random_grid = {'eta': eta,
               'max_depth': max_depth,
               'subsample': subsample,
               'number_boost_round': number_boost_round,
               'min_child_weight': min_child_weight, 
               'gamma': gamma}

combination_grid = combination_grid(random_grid)
combination_grid

TypeError: 'DataFrame' object is not callable

In [39]:
# loop whole combination grid
## loop first 10

import xgboost as xgb
from sklearn import metrics

combination_grid['avg_auc'] = np.zeros((combination_grid.shape[0],1))
combination_grid['avg_boostinground'] = np.zeros((combination_grid.shape[0],1))


for i in range(10):
## for i in range(combination_grid.shape[0]):
    auc = []
    boosting_round_list = []
    for k in range(split.shape[0]):
        train_data = train_sample_features[train_sample_features.index.isin(split.iloc[k,0])]
        validation_data = train_sample_features[train_sample_features.index.isin(split.iloc[k,1])]
        train_target = train_sample_target[train_sample_target.index.isin(split.iloc[k,0])]
        validation_target = train_sample_target[train_sample_target.index.isin(split.iloc[k,1])]

        # dMatrix
        dtrain = xgb.DMatrix(data=train_data, label=train_target)
        dtest = xgb.DMatrix(data=validation_data, label=validation_target)
        eval_sets = [(dtrain, 'train'), (dtest, 'eval')]

        
        # parameter set
        param = {'eta':combination_grid.iloc[i,0],
                 'max_depth':combination_grid.iloc[i,1],
                 'subsample':combination_grid.iloc[i,2],
                 'min_child_weight':combination_grid.iloc[i,4],
                 'gamma':combination_grid.iloc[i,5],
                 'eval_metric': 'auc'}
        #### 'eval_metric': 'auc'
        param['nthread'] = 4
        num_boost_round = combination_grid.iloc[i,3]
#         early_stopping_rounds = combination_grid.iloc[i,4] 

        # xgboot
        bst = xgb.train(param, dtrain, num_boost_round, early_stopping_rounds=50, evals=eval_sets)
        
        # preds=rf.predict(train_data)        
        ypred = bst.predict(dtest)
   
        # auc
        fpr, tpr, thr = roc_curve(validation_target, ypred)
        auc_ = metrics.auc(fpr, tpr)
        auc.append(auc_)
         
        
        # boosting round
        best_itr = bst.best_iteration
        boosting_round_list.append(best_itr)

    auc_avg = sum(auc) / len(auc)
    combination_grid.iloc[i,6] = auc_avg
    boostinground_avg = sum(boosting_round_list) / len(boosting_round_list)
    combination_grid.iloc[i,7] = boostinground_avg
    
    print('average auc: ', auc_avg, 'average boostinground: ', boostinground_avg)

[0]	train-auc:0.73622	eval-auc:0.96032
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.89152	eval-auc:0.93651
[2]	train-auc:0.93064	eval-auc:0.92063
[3]	train-auc:0.94200	eval-auc:0.92063
[4]	train-auc:0.94066	eval-auc:0.92063
[5]	train-auc:0.94278	eval-auc:0.90476
[6]	train-auc:0.94451	eval-auc:0.93651
[7]	train-auc:0.94682	eval-auc:0.92063
[8]	train-auc:0.94374	eval-auc:0.92063
[9]	train-auc:0.94374	eval-auc:0.93651
[10]	train-auc:0.94605	eval-auc:0.90476
[11]	train-auc:0.94470	eval-auc:0.90476
[12]	train-auc:0.94393	eval-auc:0.93651
[13]	train-auc:0.94220	eval-auc:0.93651
[14]	train-auc:0.94412	eval-auc:0.93651
[15]	train-auc:0.94412	eval-auc:0.93651
[16]	train-auc:0.94335	eval-auc:0.92063
[17]	train-auc:0.97245	eval-auc:0.92063
[18]	train-auc:0.97129	eval-auc:0.93651
[19]	train-auc:0.97091	eval-auc:0.93651
[20]	train-auc:0.97322	eval-auc:0.93651
[21]	train-auc:0.97283	eval-a

[58]	train-auc:0.98235	eval-auc:0.91667
[59]	train-auc:0.98198	eval-auc:0.91667
[60]	train-auc:0.98198	eval-auc:0.91667
[61]	train-auc:0.98235	eval-auc:0.91667
[62]	train-auc:0.98125	eval-auc:0.91667
[63]	train-auc:0.98162	eval-auc:0.91667
[64]	train-auc:0.98125	eval-auc:0.91667
[65]	train-auc:0.98198	eval-auc:0.93750
[66]	train-auc:0.98198	eval-auc:0.93750
Stopping. Best iteration:
[16]	train-auc:0.98217	eval-auc:0.97917

[0]	train-auc:0.71216	eval-auc:0.46875
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.81923	eval-auc:0.40625
[2]	train-auc:0.87822	eval-auc:0.37500
[3]	train-auc:0.90995	eval-auc:0.37500
[4]	train-auc:0.91474	eval-auc:0.81250
[5]	train-auc:0.91738	eval-auc:0.81250
[6]	train-auc:0.91507	eval-auc:0.87500
[7]	train-auc:0.91408	eval-auc:0.87500
[8]	train-auc:0.93589	eval-auc:0.87500
[9]	train-auc:0.93539	eval-auc:0.93750
[10]	train-auc:0.93473	eval-auc:0.93750
[1

[68]	train-auc:0.97987	eval-auc:0.97059
[69]	train-auc:0.97987	eval-auc:0.97059
[70]	train-auc:0.97987	eval-auc:0.97059
[71]	train-auc:0.97952	eval-auc:0.97059
[72]	train-auc:0.97917	eval-auc:0.97059
[73]	train-auc:0.97881	eval-auc:0.97059
[74]	train-auc:0.97881	eval-auc:0.97059
[75]	train-auc:0.97917	eval-auc:0.97059
[76]	train-auc:0.97917	eval-auc:0.97059
[77]	train-auc:0.97846	eval-auc:0.97059
[78]	train-auc:0.97917	eval-auc:0.97059
[79]	train-auc:0.97881	eval-auc:0.97059
[80]	train-auc:0.97846	eval-auc:0.97059
[81]	train-auc:0.97811	eval-auc:1.00000
[82]	train-auc:0.97811	eval-auc:1.00000
[83]	train-auc:0.97952	eval-auc:1.00000
[84]	train-auc:0.97917	eval-auc:1.00000
[85]	train-auc:0.97917	eval-auc:1.00000
[86]	train-auc:0.97881	eval-auc:1.00000
[87]	train-auc:0.97881	eval-auc:1.00000
[88]	train-auc:0.98093	eval-auc:1.00000
[89]	train-auc:0.98093	eval-auc:1.00000
[90]	train-auc:0.98058	eval-auc:1.00000
[91]	train-auc:0.98376	eval-auc:1.00000
[92]	train-auc:0.98376	eval-auc:1.00000


[20]	train-auc:0.99894	eval-auc:0.54167
[21]	train-auc:0.99822	eval-auc:0.54167
[22]	train-auc:0.99822	eval-auc:0.54167
[23]	train-auc:0.99858	eval-auc:0.54167
[24]	train-auc:0.99858	eval-auc:0.54167
[25]	train-auc:0.99858	eval-auc:0.54167
[26]	train-auc:0.99858	eval-auc:0.54167
[27]	train-auc:0.99858	eval-auc:0.54167
[28]	train-auc:0.99858	eval-auc:0.54167
[29]	train-auc:0.99929	eval-auc:0.54167
[30]	train-auc:0.99965	eval-auc:0.54167
[31]	train-auc:0.99965	eval-auc:0.54167
[32]	train-auc:0.99965	eval-auc:0.54167
[33]	train-auc:0.99965	eval-auc:0.54167
[34]	train-auc:0.99929	eval-auc:0.54167
[35]	train-auc:0.99929	eval-auc:0.54167
[36]	train-auc:0.99929	eval-auc:0.54167
[37]	train-auc:0.99929	eval-auc:0.54167
[38]	train-auc:0.99929	eval-auc:0.54167
[39]	train-auc:0.99965	eval-auc:0.58333
[40]	train-auc:0.99965	eval-auc:0.58333
[41]	train-auc:0.99965	eval-auc:0.58333
[42]	train-auc:0.99965	eval-auc:0.58333
[43]	train-auc:0.99965	eval-auc:0.58333
[44]	train-auc:0.99965	eval-auc:0.58333


[1]	train-auc:0.87571	eval-auc:0.97059
[2]	train-auc:0.87606	eval-auc:0.97059
[3]	train-auc:0.87871	eval-auc:0.94118
[4]	train-auc:0.92514	eval-auc:0.94118
[5]	train-auc:0.92638	eval-auc:0.94118
[6]	train-auc:0.92002	eval-auc:0.94118
[7]	train-auc:0.91896	eval-auc:0.94118
[8]	train-auc:0.91861	eval-auc:0.94118
[9]	train-auc:0.91878	eval-auc:0.94118
[10]	train-auc:0.91861	eval-auc:0.94118
[11]	train-auc:0.91755	eval-auc:0.94118
[12]	train-auc:0.91720	eval-auc:0.94118
[13]	train-auc:0.91684	eval-auc:0.94118
[14]	train-auc:0.91649	eval-auc:0.94118
[15]	train-auc:0.91631	eval-auc:0.94118
[16]	train-auc:0.94862	eval-auc:0.91177
[17]	train-auc:0.96257	eval-auc:0.91177
[18]	train-auc:0.96257	eval-auc:0.94118
[19]	train-auc:0.96222	eval-auc:0.94118
[20]	train-auc:0.97210	eval-auc:0.94118
[21]	train-auc:0.97210	eval-auc:0.94118
[22]	train-auc:0.97210	eval-auc:0.94118
[23]	train-auc:0.97564	eval-auc:0.91177
[24]	train-auc:0.97564	eval-auc:0.91177
[25]	train-auc:0.97564	eval-auc:0.94118
[26]	trai

...

[31]	train-auc:0.98095	eval-auc:1.00000
[32]	train-auc:0.98095	eval-auc:1.00000
[33]	train-auc:0.98062	eval-auc:1.00000
[34]	train-auc:0.97961	eval-auc:1.00000
[35]	train-auc:0.97794	eval-auc:1.00000
[36]	train-auc:0.97894	eval-auc:1.00000
[37]	train-auc:0.98095	eval-auc:1.00000
[38]	train-auc:0.98128	eval-auc:1.00000
[39]	train-auc:0.98095	eval-auc:1.00000
[40]	train-auc:0.98062	eval-auc:1.00000
[41]	train-auc:0.98028	eval-auc:1.00000
[42]	train-auc:0.98028	eval-auc:1.00000
[43]	train-auc:0.98028	eval-auc:1.00000
[44]	train-auc:0.98062	eval-auc:1.00000
[45]	train-auc:0.98062	eval-auc:1.00000
[46]	train-auc:0.98028	eval-auc:1.00000
[47]	train-auc:0.97995	eval-auc:1.00000
[48]	train-auc:0.98028	eval-auc:1.00000
[49]	train-auc:0.97961	eval-auc:1.00000
[50]	train-auc:0.97995	eval-auc:1.00000
[51]	train-auc:0.97928	eval-auc:1.00000
Stopping. Best iteration:
[1]	train-auc:0.88235	eval-auc:1.00000

[0]	train-auc:0.81378	eval-auc:0.41304
Multiple eval metrics have been passed: 'eval-auc' will

[56]	train-auc:0.98240	eval-auc:0.92500
[57]	train-auc:0.98204	eval-auc:0.92500
[58]	train-auc:0.98312	eval-auc:0.92500
[59]	train-auc:0.98312	eval-auc:0.92500
[60]	train-auc:0.98743	eval-auc:0.92500
[61]	train-auc:0.98707	eval-auc:0.92500
[62]	train-auc:0.98851	eval-auc:0.92500
[63]	train-auc:0.98851	eval-auc:0.95000
[64]	train-auc:0.98851	eval-auc:0.95000
Stopping. Best iteration:
[14]	train-auc:0.97540	eval-auc:0.95000

average auc:  0.8075015223480697 average boostinground:  16.7
[0]	train-auc:0.73622	eval-auc:0.96032
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.89152	eval-auc:0.93651
[2]	train-auc:0.93064	eval-auc:0.92063
[3]	train-auc:0.93738	eval-auc:0.92063
[4]	train-auc:0.93738	eval-auc:0.92063
[5]	train-auc:0.94104	eval-auc:0.90476
[6]	train-auc:0.94162	eval-auc:0.90476
[7]	train-auc:0.94239	eval-auc:0.88889
[8]	train-auc:0.94085	eval-auc:0.88889
[9]	train-auc:0.941

[45]	train-auc:0.97537	eval-auc:0.89583
[46]	train-auc:0.97574	eval-auc:0.91667
[47]	train-auc:0.97574	eval-auc:0.89583
[48]	train-auc:0.97574	eval-auc:0.89583
[49]	train-auc:0.97537	eval-auc:0.89583
[50]	train-auc:0.97647	eval-auc:0.89583
[51]	train-auc:0.97721	eval-auc:0.89583
[52]	train-auc:0.97647	eval-auc:0.89583
[53]	train-auc:0.97647	eval-auc:0.89583
[54]	train-auc:0.97610	eval-auc:0.89583
[55]	train-auc:0.97610	eval-auc:0.89583
[56]	train-auc:0.97610	eval-auc:0.89583
[57]	train-auc:0.97610	eval-auc:0.89583
[58]	train-auc:0.97610	eval-auc:0.89583
Stopping. Best iteration:
[8]	train-auc:0.93824	eval-auc:0.93750

[0]	train-auc:0.71216	eval-auc:0.46875
Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.79362	eval-auc:0.43750
[2]	train-auc:0.87855	eval-auc:0.40625
[3]	train-auc:0.91028	eval-auc:0.40625
[4]	train-auc:0.91309	eval-auc:0.87500
[5]	train-auc:0.91573	eval-auc:0.87500

[62]	train-auc:0.97564	eval-auc:0.97059
[63]	train-auc:0.97564	eval-auc:0.97059
[64]	train-auc:0.97493	eval-auc:0.97059
[65]	train-auc:0.97493	eval-auc:0.97059
[66]	train-auc:0.97493	eval-auc:0.97059
[67]	train-auc:0.97493	eval-auc:0.97059
[68]	train-auc:0.97493	eval-auc:0.97059
[69]	train-auc:0.97493	eval-auc:0.97059
[70]	train-auc:0.97422	eval-auc:0.97059
[71]	train-auc:0.97422	eval-auc:0.97059
[72]	train-auc:0.97881	eval-auc:0.97059
[73]	train-auc:0.97881	eval-auc:0.97059
[74]	train-auc:0.97881	eval-auc:0.97059
[75]	train-auc:0.97881	eval-auc:0.97059
[76]	train-auc:0.97881	eval-auc:0.97059
[77]	train-auc:0.97846	eval-auc:0.97059
[78]	train-auc:0.97846	eval-auc:0.97059
[79]	train-auc:0.97846	eval-auc:0.97059
[80]	train-auc:0.97846	eval-auc:0.97059
[81]	train-auc:0.97775	eval-auc:0.97059
[82]	train-auc:0.97811	eval-auc:0.97059
[83]	train-auc:0.97952	eval-auc:0.97059
[84]	train-auc:0.97952	eval-auc:0.97059
[85]	train-auc:0.97987	eval-auc:0.97059
[86]	train-auc:0.97952	eval-auc:0.97059


Multiple eval metrics have been passed: 'eval-auc' will be used for early stopping.

Will train until eval-auc hasn't improved in 50 rounds.
[1]	train-auc:0.86950	eval-auc:0.65278
[2]	train-auc:0.90021	eval-auc:0.65278
[3]	train-auc:0.89719	eval-auc:0.65278
[4]	train-auc:0.96484	eval-auc:0.59722
[5]	train-auc:0.98224	eval-auc:0.59722
[6]	train-auc:0.98313	eval-auc:0.59722
[7]	train-auc:0.98029	eval-auc:0.59722
[8]	train-auc:0.98136	eval-auc:0.59722
[9]	train-auc:0.98739	eval-auc:0.59722
[10]	train-auc:0.98704	eval-auc:0.59722
[11]	train-auc:0.98704	eval-auc:0.59722
[12]	train-auc:0.98597	eval-auc:0.59722
[13]	train-auc:0.98846	eval-auc:0.56944
[14]	train-auc:0.98633	eval-auc:0.55556
[15]	train-auc:0.98704	eval-auc:0.56944
[16]	train-auc:0.98952	eval-auc:0.56944
[17]	train-auc:0.98917	eval-auc:0.56944
[18]	train-auc:0.99379	eval-auc:0.56944
[19]	train-auc:0.99414	eval-auc:0.51389
[20]	train-auc:0.99556	eval-auc:0.51389
[21]	train-auc:0.99556	eval-auc:0.54167
[22]	train-auc:0.99521	eval-

In [40]:
# # train test split and retain test data for final evaluation
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

# # split train set again into train and validation set
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

# Dtrain = xgb.DMatrix(X_train, y_train)  
# Deval = xgb.DMatrix(X_val, y_val)
# eval_sets = [(Dtrain, 'train'), (Deval, 'eval')]
# model = xgb.train(params_booster, Dtrain, 
#                   verbose_eval=verbose_eval_, evals=eval_sets,        
#                   early_stopping_rounds=10, evals_result=progress, 
#                   num_boost_round = 250)

#### eval_list ref. https://stackoverflow.com/questions/43062613/how-to-randomly-select-rows-from-a-data-set-using-pandas

In [45]:
combination_grid.head(10).sort_values(by='avg_auc', ascending=False)

,eta,max_depth,subsample,number_boost_round,min_child_weight,gamma,avg_auc,avg_boostinground
0,0.01,4,0.6,500,0.1,0.0,0.807502,16.7
2,0.01,4,0.6,500,0.2,0.0,0.807502,16.7
4,0.01,4,0.6,500,0.3,0.0,0.807502,16.7
6,0.01,4,0.6,500,0.4,0.0,0.807502,16.7
8,0.01,4,0.6,500,0.5,0.0,0.807502,16.7
1,0.01,4,0.6,500,0.1,0.5,0.804380,17.3
3,0.01,4,0.6,500,0.2,0.5,0.804380,17.3
5,0.01,4,0.6,500,0.3,0.5,0.804380,17.3
7,0.01,4,0.6,500,0.4,0.5,0.804380,17.3
9,0.01,4,0.6,500,0.5,0.5,0.804380,17.3


In [52]:
## distribution of 


# import numpy as np
# import seaborn as sns
# data = [1.5]*7 + [2.5]*2 + [3.5]*8 + [4.5]*3 + [5.5]*1 + [6.5]*8
# sns.set_style('whitegrid')
# sns.kdeplot(np.array(data), bw=0.5)


### ref. https://stackoverflow.com/questions/4150171/how-to-create-a-density-plot-in-matplotlib